In [1]:
import sys
sys.path.append("/home/jkdamewo/utils")

from imports import *
from django_imports import *
from datetime import datetime
import time
WORKDIR = os.getcwd()

In [2]:
group = Group.objects.get(name='HighEntropyAlloys')
config = JobConfig.objects.filter(name__in=['bcc_supercell'])[0]

In [3]:
crys_parent = Crystal.objects.filter(
        parentjob__group=group,
        parentjob__config=config,
).distinct()

In [4]:
print(len(crys_parent))

5


In [5]:
def traverse_opt_dft(parent_crystals):
    child_ids = []
    unrelaxed_ids = {}
    for parent in tqdm(parent_crystals):
        try:
            child = parent.childjobs.get(
                status='done',
                config__name='pbe_u_paw_spinpol_opt_vasp'
            ).childgeoms.get(
                converged=True
            ).crystal
        except (Job.DoesNotExist, Geom.DoesNotExist):
            pass
        else:
            child_ids.append(child.id)

    return child_ids

In [6]:
def traverse_es_dft(crystals ):
    child_ids = []
    es_jobs = []
    unrelaxed_ids = []
    count = 0
    for cryst in tqdm(crystals):
        try:
            es_job = cryst.childjobs.get(status='done',
                        config__name='pbe_u_paw_spinpol_es_vasp')
        except (Job.DoesNotExist,
                Geom.DoesNotExist,
                django.core.exceptions.MultipleObjectsReturned):
            pass
        else:
            child_ids.append(cryst.id)
    
    return child_ids

In [7]:
conv_crys = traverse_opt_dft(crys_parent)

100%|██████████| 5/5 [00:00<00:00, 150.29it/s]


In [36]:
crys_parent[4].childjobs.get().__dict__

{'_state': <django.db.models.base.ModelState at 0x7f12ffe4d190>,
 'id': 123200473,
 'uuid': UUID('e8a7053f-1656-4548-959b-7a8be70581d7'),
 'config_id': 252,
 'group_id': 64,
 'priority': 1,
 'workbatch_id': None,
 'status': 'done',
 'createtime': datetime.datetime(2023, 6, 21, 21, 34, 45, 48008, tzinfo=datetime.timezone.utc),
 'parentct_id': 54,
 'parentid': 425307767,
 'details': {'imix': 4,
  'isym': 2,
  'kpar': 1,
  'ldau': True,
  'ediff': 1e-06,
  'encut': 520,
  'ispin': 2,
  'lreal': 'Auto',
  'ncore': 4,
  'potim': 0.5,
  'ediffg': -0.01,
  'ibrion': 2,
  'ismear': 0,
  'kppvol': 64,
  'memory': 394.46,
  'nelmin': 4,
  'nprocs': 16,
  'nsteps': 300,
  'kpoints': [2, 2, 2],
  'lattice': [[9.797958975728635, 0.0, -3.46410161589873],
   [-4.8989794862352785, 8.48528137915928, -3.46410161589873],
   [0.0, 0.0, 10.39230485]],
  'lmaxmix': 2,
  'magmoms': None,
  'symprec': 1e-05,
  'version': '5.4.4.18Apr17-6-g9f103f2a35',
  'fix_cell': False,
  'ldautype': 2,
  'nodename': 'node1

In [8]:
done_crys = traverse_es_dft(conv_crys)

0it [00:00, ?it/s]


In [9]:
import os
from pymatgen.io.cif import CifWriter
from pymatgen.transformations.site_transformations import RemoveSitesTransformation

In [14]:
parent_path = "V4Cr4Ti/"
if not os.path.exists(parent_path):
    os.mkdir(parent_path)

In [ ]:
def find_new_loc(parent,loc,direction):
    best_site = None
    best_val = 999999999
    positions = parent.frac_coords
    scale = ((parent.volume)**(1/3))/4.0
    
    for i in range(positions.shape[0]):
        
        curr_site = i
        loc_positions = positions[loc,:]
        
        
        if direction == [1,0,0]:
            bcc_direction = np.asarray([1.0,0.0,0.0])
        elif direction == [0,1,0]:
            bcc_direction = np.asarray([0.0,1.0,0.0])
        elif direction == [0,0,1]:
            bcc_direction = np.asarray([0.0,0.0,1.0])
        elif direction == [1,1,1]
            bcc_direction = np.asarray([0.5,0.5,0.5])
        elif direction == [-1,1,1]
            bcc_direction = np.asarray([-0.5,0.5,0.5])
        else:
            print("Direction Not Defined")
        proposed_positions = loc_positions + (0.25)*bcc_direction
        
        
        for j in range(proposed_positions.shape[0]):
            if proposed_positions[j]>1.0:
                proposed_positions[j]-=1
        
        curr_positions = positions[i,:]
        
        diff = proposed_positions - curr_positions
        
        curr_val = np.sum(diff**2)
        
    
        if curr_val < best_val:
            best_val = curr_val
            best_site = curr_site
            
    return best_site
            
    
    

In [ ]:
def select_vacancies(parent,loc,path):
    if not os.path.exists(path):
        os.mkdir(path)
        
    if not os.path.exists(path+"000/"):
        os.mkdir(path+"000/")
        
    trans = RemoveSitesTransformation(loc)
    struct_000 = trans.apply_transformation(parent)
    writer_000 = CifWriter(struct_000)
    writer_000.write_file(path+"000/"+"vacancy_center.cif")
    
    ###################
    
    
    if not os.path.exists(path+"100/"):
        os.mkdir(path+"100/")
        
    end_loc = find_new_loc(parent,loc,[1,0,0])
    trans = RemoveSitesTransformation(end_loc)
    struct_100 = trans.apply_transformation(parent)
    
    writer_100 = CifWriter(struct_100)
    writer_100.write_file(path+"100/"+"end.cif")
    writer_000.write_file(path+"100/"+"start.cif")
    
    ##################
    
    if not os.path.exists(path+"010/"):
        os.mkdir(path+"010/")
        
    end_loc = find_new_loc(parent,loc,[0,1,0])
    trans = RemoveSitesTransformation(end_loc)
    struct_010 = trans.apply_transformation(parent)
    
    writer_010 = CifWriter(struct_010)
    writer_010.write_file(path+"010/"+"end.cif")
    writer_000.write_file(path+"010/"+"start.cif")
    
    ##########################
    
    if not os.path.exists(path+"001/"):
        os.mkdir(path+"001/")
        
    end_loc = find_new_loc(parent,loc,[0,0,1])
    trans = RemoveSitesTransformation(end_loc)
    struct_001 = trans.apply_transformation(parent)
    
    writer_001 = CifWriter(struct_001)
    writer_001.write_file(path+"001/"+"end.cif")
    writer_000.write_file(path+"001/"+"start.cif")
    
    ########################
    
    if not os.path.exists(path+"111/"):
        os.mkdir(path+"111/")
        
    end_loc = find_new_loc(parent,loc,[1,1,1])
    trans = RemoveSitesTransformation(end_loc)
    struct_111 = trans.apply_transformation(parent)
    
    writer_111 = CifWriter(struct_111)
    writer_111.write_file(path+"111/"+"end.cif")
    writer_000.write_file(path+"111/"+"start.cif")
    
    ######################################
    
    if not os.path.exists(path+"-111/"):
        os.mkdir(path+"-111/")
        
    end_loc = find_new_loc(parent,loc,[-1,1,1])
    trans = RemoveSitesTransformation(end_loc)
    struct_m111 = trans.apply_transformation(parent)
    
    writer_m111 = CifWriter(struct_m111)
    writer_m111.write_file(path+"m111/"+"end.cif")
    writer_000.write_file(path+"m111/"+"start.cif")
    

In [15]:
import numpy as np

def create_vacancies_from_structure(crystal,path):
    if not os.path.exists(path):
        os.mkdir(path)
        
    curr_parent = crystal.as_pymatgen_structure()
    parent_writer = CifWriter(curr_parent)
    parent_writer.write_file(path+"parent.cif")
    
    sites = np.random.choice(curr_parent.frac_coords.shape[0], 5, replace = False)
    
    for i in range(5):
        
        select_vacancies(curr_parent,sites[i],path+"vacancy_num"+str(i)+"/")

SyntaxError: invalid syntax (2569028013.py, line 7)

In [ ]:
for i in range(done_crys):
    create_vacancies_from_structure(done_crys[i],parent_path+"Structure_Num" + str(i) + "/")